In [36]:
### Importing libraries
import json
import os
import glob
import dill
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup
import requests

In [2]:
ruralDF = pd.read_excel("https://github.com/charlottehchang/BIOL104PO/blob/master/data/ruralurbancodes2013.xls?raw=true")

In [3]:
ruralDF

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description
0,1001,AL,Autauga County,54571,2,"Metro - Counties in metro areas of 250,000 to ..."
1,1003,AL,Baldwin County,182265,3,Metro - Counties in metro areas of fewer than ...
2,1005,AL,Barbour County,27457,6,"Nonmetro - Urban population of 2,500 to 19,999..."
3,1007,AL,Bibb County,22915,1,Metro - Counties in metro areas of 1 million p...
4,1009,AL,Blount County,57322,1,Metro - Counties in metro areas of 1 million p...
...,...,...,...,...,...,...
3216,72145,PR,Vega Baja Municipio,59662,1,Metro - Counties in metro areas of 1 million p...
3217,72147,PR,Vieques Municipio,9301,7,"Nonmetro - Urban population of 2,500 to 19,999..."
3218,72149,PR,Villalba Municipio,26073,2,"Metro - Counties in metro areas of 250,000 to ..."
3219,72151,PR,Yabucoa Municipio,37941,1,Metro - Counties in metro areas of 1 million p...


In [21]:
ruralDF['FIPSstr'] = ruralDF.FIPS.apply(str) # ruralDF$FIPS; dplyr::select(ruralDF, FIPS)

In [22]:
ruralDF.FIPSstr

0        1001
1        1003
2        1005
3        1007
4        1009
        ...  
3216    72145
3217    72147
3218    72149
3219    72151
3220    72153
Name: FIPSstr, Length: 3221, dtype: object

In [23]:
ruralDF.FIPSstr.str.slice(-3).iloc[:10]

0    001
1    003
2    005
3    007
4    009
5    011
6    013
7    015
8    017
9    019
Name: FIPSstr, dtype: object

In [24]:
ruralDF['CtyFIPS'] = 'US-' + ruralDF.State + "-" + ruralDF.FIPSstr.str.slice(-3)

In [25]:
ruralDF.CtyFIPS

0       US-AL-001
1       US-AL-003
2       US-AL-005
3       US-AL-007
4       US-AL-009
          ...    
3216    US-PR-145
3217    US-PR-147
3218    US-PR-149
3219    US-PR-151
3220    US-PR-153
Name: CtyFIPS, Length: 3221, dtype: object

In [29]:
ruralDF.State.unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR'], dtype=object)

In [34]:
### Remove Puerto Rico
ruralDF48 = ruralDF[ruralDF.State!="PR"] # dplyr::filter

In [35]:
ruralDF48

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description,FIPSstr,CtyFIPS
0,1001,AL,Autauga County,54571,2,"Metro - Counties in metro areas of 250,000 to ...",1001,US-AL-001
1,1003,AL,Baldwin County,182265,3,Metro - Counties in metro areas of fewer than ...,1003,US-AL-003
2,1005,AL,Barbour County,27457,6,"Nonmetro - Urban population of 2,500 to 19,999...",1005,US-AL-005
3,1007,AL,Bibb County,22915,1,Metro - Counties in metro areas of 1 million p...,1007,US-AL-007
4,1009,AL,Blount County,57322,1,Metro - Counties in metro areas of 1 million p...,1009,US-AL-009
...,...,...,...,...,...,...,...,...
3138,56037,WY,Sweetwater County,43806,5,"Nonmetro - Urban population of 20,000 or more,...",56037,US-WY-037
3139,56039,WY,Teton County,21294,7,"Nonmetro - Urban population of 2,500 to 19,999...",56039,US-WY-039
3140,56041,WY,Uinta County,21118,7,"Nonmetro - Urban population of 2,500 to 19,999...",56041,US-WY-041
3141,56043,WY,Washakie County,8533,7,"Nonmetro - Urban population of 2,500 to 19,999...",56043,US-WY-043


In [38]:
time_period="cur"

In [49]:
ruralDF48.CtyFIPS.iloc[76:]

76      US-AK-105
77      US-AK-110
78      US-AK-122
79      US-AK-130
80      US-AK-150
          ...    
3138    US-WY-037
3139    US-WY-039
3140    US-WY-041
3141    US-WY-043
3142    US-WY-045
Name: CtyFIPS, Length: 3067, dtype: object

In [76]:
eBirdOut = [] # eBirdOut = c()
for ctyFIP in ruralDF48.CtyFIPS:
    eBirdURL = f"https://ebird.org/region/{ctyFIP}?yr={time_period}"
    r=requests.get(eBirdURL).text
    soup=BeautifulSoup(r)
    species_observed = soup.select_one("span[id*=speciesStat]")
    if species_observed is not None:
        try:
            species_observed = int(species_observed.text.strip())
        except:
            species_observed = species_observed
    else:
        species_observed = -999
    checklists = soup.select_one("span[id*=checklistStat]")
    if checklists is not None:
        try:
            checklists = int(checklists.text.strip().replace(",",""))
        except:
            checklists = checklists
    else:
        checklists = -999
    hotspots = soup.select_one("span[id*=hotspotStat]")
    if hotspots is not None:
        try:
            hotspots = int(hotspots.text.strip())
        except:
            hotspots = hotspots
    else:
        hotspots = -999
    eBirderCt = soup.select_one("span[id*=ebirdersStat]")
    if eBirderCt is not None:
        try:
            eBirderCt = int(eBirderCt.text.strip().replace(',',''))
        except:
            eBirderCt = eBirderCt
    else:
        eBirderCt = -999
    eBirdEntry={'Species':species_observed,'Checklists':checklists,'Hotspots':hotspots,'eBirders':eBirderCt}
    eBirdOut.append(eBirdEntry)

In [69]:
eBirdURL

'https://ebird.org/region/US-AZ-013?yr=cur'

In [57]:
checklists

12978

In [70]:
hotspots.text.strip()

'12,978'

In [59]:
hotspots.select_one("span[id*=compact-number]")

In [82]:
len(eBirdOut)

3143

In [52]:
ctyFIP

'US-AK-105'

In [77]:
eBirdDF = pd.DataFrame(eBirdOut)

In [78]:
eBirdDF['FIPS'] = ruralDF48.FIPS
eBirdDF['State'] = ruralDF48.State
eBirdDF['County'] = ruralDF48.County_Name.str.replace(' County',"")

In [79]:
eBirdDF.to_csv('eBirdData.csv',index=False)

In [81]:
! tail -10 eBirdData.csv

12,2,3,292,56027,WY,Niobrara
98,396,164,4326,56029,WY,Park
66,29,15,500,56031,WY,Platte
74,287,32,688,56033,WY,Sheridan
73,189,30,621,56035,WY,Sublette
66,117,42,1104,56037,WY,Sweetwater
94,509,208,6009,56039,WY,Teton
57,43,15,616,56041,WY,Uinta
62,224,18,228,56043,WY,Washakie
68,56,10,363,56045,WY,Weston


In [75]:
eBirdDF

,Species,Checklists,Hotspots,eBirders,FIPS,State,County
0,104,153,6,197,1001,AL,Autauga
1,243,2256,97,2763,1003,AL,Baldwin
2,147,179,16,556,1005,AL,Barbour
3,93,93,9,188,1007,AL,Bibb
4,52,16,4,165,1009,AL,Blount
5,98,97,2,161,1011,AL,Bullock
6,53,12,3,208,1013,AL,Butler
7,126,310,17,270,1015,AL,Calhoun
8,91,35,11,225,1017,AL,Chambers
9,73,66,6,180,1019,AL,Cherokee
